In [ ]:
from dotenv import load_dotenv
from agent import Agent
load_dotenv()

react = Agent(llm="gpt-4o-mini", system_prompt="I am a helpful AI assistant.")
# react = Agent(llm="llama 3.3 70B", system_prompt="I am a helpful AI assistant.")
response = react.generate_response(prompt="What is the capital of Taiwan?")
print(response)

In [ ]:
from tools import search

results = search("128k context window llms?", max_results=20)

print(results)

In [ ]:
print(results[:10])

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("o200k_base")

enc = tiktoken.encoding_for_model("gpt-4o")

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

print(len(enc.encode(str(results[:10]))))

In [ ]:
from agent import Agent
from dotenv import load_dotenv

load_dotenv()

react_system_prompt = """You are an AI Agent based on the ReAct framework, tasked with answering questions through the Observe -> Reason -> Act loop.

## **Action Rules**
1. **Reason**  
   - First, think about the question logically using available information.  
   - Explain your reasoning in natural language before taking action.  

2. **Act**  
   - If more information is needed, use the **search tool** to retrieve relevant information.  
   - If enough information has been gathered, use the **final_answer tool** to generate the final response.  

### **Available Tools**
- `search(query: str) -> List[Dict]`  
  - Use this tool when additional information is needed.  
  - Example: _"I should search for 'Who discovered dark energy?' to find relevant information."_

- `final_answer() -> str`  
  - Use this tool when you have gathered enough information to provide a final answer.  
  - Example: _"I now have enough information to answer the question."_

## **Observation Rules**
- When you receive `<retrieved data>`, it contains search results.  
- Use these search results to improve your reasoning before taking further action.

## **Examples**

### **Searching for Information**
'''
Based on the question, I need to search for "Who discovered dark energy?" to get background knowledge.
'''
_(You should use the `search` tool in this case.)_

### **Answering After Obtaining Sufficient Information**
'''
Based on the retrieved data, I now have enough information to answer the question.
'''
_(You should use the `final_answer` tool in this case.)_
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Retrieve relevant web search results for a given query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query string."
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    }, {
        "type": "function",
        "function": {
            "name": "final_answer",
            "description": "Generate a final answer based on the conversation history.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    }
]

p = [{"role": "system", "content": react_system_prompt},
     {"role": "user", "content": f"how did people prepare for typhoon haiyan"}
     ]

# messages + conversations

ag = Agent(llm="llama 3.3 70B")
message = ag.generate_response(conversations=p, tools=tools)
print(message.content)
print("---")
print(message.tool_calls)

In [ ]:
print(message)

In [ ]:
from agent import Agent
from dotenv import load_dotenv

load_dotenv()

react_system_prompt = """You are an AI Agent based on the ReAct framework, tasked with answering questions through the Observe -> Reason -> Act loop.

## **Action Rules**
1. **Reason**  
   - First, think about the question logically using available information.  
   - Explain your reasoning in natural language before taking action.  

2. **Act**  
   - If more information is needed, use the **search tool** to retrieve relevant information.  
   - If enough information has been gathered, use the **final_answer tool** to generate the final response.  

### **Available Tools**
- `search(query: str) -> List[Dict]`  
  - Use this tool when additional information is needed.  
  - Example: _"I should search for 'Who discovered dark energy?' to find relevant information."_

- `final_answer() -> str`  
  - Use this tool when you have gathered enough information to provide a final answer.  
  - Example: _"I now have enough information to answer the question."_

## **Observation Rules**
- When you receive `<retrieved data>`, it contains search results.  
- Use these search results to improve your reasoning before taking further action.

## **Examples**

### **Searching for Information**
'''
Based on the question, I need to search for "Who discovered dark energy?" to get background knowledge.
'''
_(You should use the `search` tool in this case.)_

### **Answering After Obtaining Sufficient Information**
'''
Based on the retrieved data, I now have enough information to answer the question.
'''
_(You should use the `final_answer` tool in this case.)_
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Retrieve relevant web search results for a given query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query string."
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    }, {
        "type": "function",
        "function": {
            "name": "final_answer",
            "description": "Generate a final answer based on the conversation history.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    }
]
p2 = [{"role": "developer", "content": react_system_prompt},
     {"role": "user", "content": f"how did people prepare for typhoon haiyan"}
     ]

ag2 = Agent(llm="gpt-4o-mini")
message2, usage2 = ag2.generate_response(conversations=p2, tools=tools)
print(message2)
print("---")
print(usage2)

In [ ]:
import json
print(message2.tool_calls[0].function)
function_name = message2.tool_calls[0].function.name
arguments = json.loads(message2.tool_calls[0].function.arguments)

print(function_name)
print(arguments)